In [1]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/side_concordance_results.xlsx')

In [2]:
def extract_side(location):
    if pd.isnull(location):
        return 'No Scan'
    
    if 'Left' in location:
        return 'Left'
    elif 'Right' in location:
        return 'Right'
    else:
        return 'Unknown'
    
preop_imaging_df['Surgical Side'] = preop_imaging_df['Surgical_Findings'].apply(extract_side)

In [3]:
def check_spect_sestamibi(row):
    spect = row['SPECT/CT']
    sestamibi = row['Sestamibi']
    
    spect_localizing = pd.notnull(spect) and spect not in ['No Scan', 'Non-localizing']
    sestamibi_localizing = pd.notnull(sestamibi) and sestamibi not in ['No Scan', 'Non-localizing']
    
    if not pd.notnull(spect) and not pd.notnull(sestamibi):
        return 'No SPECT/CT or Sestamibi Recorded'
    elif spect_localizing and sestamibi_localizing:
        return f"SPECT: {spect} | Sestamibi: {sestamibi}"
    elif spect_localizing:
        return f'SPECT/CT: {spect}'
    elif sestamibi_localizing:
        return f'Sestamibi: {sestamibi}'
    else:
        return 'Non-localizing'
    
preop_imaging_df['SPECT/CT-Sestamibi'] = preop_imaging_df.apply(check_spect_sestamibi, axis=1)
preop_imaging_df['SPECT/CT-Sestamibi'].head()

0    Sestamibi: Right Inferior
1               Non-localizing
2     Sestamibi: Left Inferior
3     Sestamibi: Left Inferior
4              Sestamibi: Left
Name: SPECT/CT-Sestamibi, dtype: object

In [4]:
# Define modality
modality = 'SPECT/CT-Sestamibi'

# Apply extract_side function if not already done to get imaging sides
preop_imaging_df['SPECT/CT-Sestamibi Imaging Side'] = preop_imaging_df[modality].apply(extract_side)

# Filter to scans that localized anywhere (Left, Right)
localized_spectmibi = preop_imaging_df[preop_imaging_df['SPECT/CT-Sestamibi Imaging Side'].isin(['Left', 'Right'])].copy()

# Count total localized scans
total_localized_spectmibi = localized_spectmibi.shape[0]
total_localized_spectmibi

285

In [5]:
def check_concordance(row):
    imaging = row['SPECT/CT-Sestamibi']
    surgical = row['Surgical_Findings']
    
    if pd.isnull(imaging) or pd.isnull(surgical):
        return 'Unknown'
    imaging_glands = [g.strip() for g in imaging.split(',')]
    surgical_glands = [g.strip() for g in surgical.split(',')]
    
    if any(sg in imaging_glands for sg in surgical_glands):
        return 'Yes (Exact)'
    
    imaging_sides = []
    for ig in imaging_glands:
        if 'Left' in ig:
            imaging_sides.append('Left')
        if 'Right' in ig:
            imaging_sides.append('Right')
    
    surgical_sides = []
    for sg in surgical_glands:
        if 'Left' in sg:
            surgical_sides.append('Left')
        if 'Right' in sg:
            surgical_sides.append('Right')
    
    if any(side in surgical_sides for side in imaging_sides):
        return 'Yes (Side)'
    
    return 'Incorrect Localization'


In [6]:
localized_spectmibi['Calculated Concordance'] = localized_spectmibi.apply(check_concordance, axis=1)

false_positives = (localized_spectmibi['Calculated Concordance'] == 'Incorrect Localization').sum()
true_positive = (
    (localized_spectmibi['Calculated Concordance'] == 'Yes (Exact)') |
    (localized_spectmibi['Calculated Concordance'] == 'Yes (Side)')
).sum()

ppv = true_positive / (true_positive + false_positives)

In [7]:
print(f"Positive Predictive Value (PPV) for localized SPECT/CT + Sestamibi scans: {ppv:.2%}")

Positive Predictive Value (PPV) for localized SPECT/CT + Sestamibi scans: 94.74%
